In [1]:
import kmapper as km
from kmapper.plotlyviz import *
import sklearn
#from sklearn import ensemble
# DBSCAN from sklearn for clustering algorithms
from sklearn.cluster import DBSCAN
# PCA from sklearn for projection/lens creation
from sklearn.decomposition import PCA
# from sklearn.manifold import MDS

# scipy for interpolation
# import scipy 
# from scipy.interpolate import *
# Plotly and Dash
import plotly.graph_objs as go
from kmapper.plotlyviz import plotlyviz
from kmapper.plotlyviz import *
from ipywidgets import (HBox, VBox)
import dash_html_components as html
import dash_core_components as dcc
import dash
from ipywidgets import interactive, HBox, VBox, widgets, interact
import warnings
import pandas as pd
import json
warnings.filterwarnings("ignore")

In [2]:
shape_df = pd.read_csv(r"C:\Users\killiad\Documents\Senior\REU\Mapper\PCA_1_NO_COND\shape.csv")
jsonFilePath = r"C:\Users\killiad\Documents\Senior\REU\Mapper\PCA_1_NO_COND\TDA1PCA15Cubes35perc.json"
jsonFile = open(jsonFilePath, "r")
jsonData = json.load(jsonFile) 
jsonFile.close()

scomplex = jsonData["['Stratum 3 SUMMER 14-20: ']"]

In [10]:
def cluster_fun(dict_df, DBSCAN_EPSILON = 20, DBSCAN_MIN_SAMPLES = 1, N_CUBES = [7,7], PERC_OVERLAP = [.5,.5]):
    """

    """
    
    
    keys = list(dict_df.keys())
    print(keys)
    X = dict_df.get(keys[0])
    X = X[["WDP", "SECCHI", "TEMP", "DO", 
           "TURB", "VEL", "TP", 
           "TN", "SS", "CHLcal"]]
    if X.shape[0]<DBSCAN_MIN_SAMPLES:
        #print(X)
        print("Not enough data to cluster in ", keys, "_size = ", X.shape[0])
        print("DBSCAN_MIN_SAMPLES", DBSCAN_MIN_SAMPLES)
        return([DBSCAN_MIN_SAMPLES, X.shape[0]])
    
    
    db = DBSCAN(eps=20, min_samples=2).fit(X)
    # Number of clusters in labels, ignoring noise if present.
    labels = db.labels_
    n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)
    n_noise_ = list(labels).count(-1)

    print('Estimated number of clusters: %d' % n_clusters_)
    print('Estimated number of noise points: %d' % n_noise_)
    
    return(db)

def mapper(scomplex, shape_df):
    """
    str(df_stratum_season_time_dict_list[1].keys())
    """
    
    continuous_variables = ["PredictedWDP", "PredictedSECCHI", "PredictedTEMP", "PredictedDO", 
           "PredictedTURB", "PredictedVEL", "PredictedTP", 
           "PredictedTN", "PredictedSS", "PredictedCHLcal", "COLOR"]
    var_to_index = {continuous_variables[i] : i for i in range(len(continuous_variables))}
    shape_df = shape_df[continuous_variables]
    shape_df.reset_index(drop = True, inplace = True)
    
    projected_vars = continuous_variables
    #projected_vars = ['PredictedTP, PredictedTN']
    projected_var_indices = [var_to_index[var] for var in projected_vars]

    # Instantiate kepler mapper object
    mapper = km.KeplerMapper(verbose=0)
    

    #lens = np.array(X[continuous_variables[max_index]])
    #lens = np.array(X[['PredictedTN', 'PredictedSS']])
    # Generate the simplicial complex
    summary_variable = mapper.project(np.array(shape_df), projection=projected_var_indices, scaler=None)

    pl_brewer = [[0.0, '#006837'],
             [0.1, '#1a9850'],
             [0.2, '#66bd63'],
             [0.3, '#a6d96a'],
             [0.4, '#d9ef8b'],
             [0.5, '#ffffbf'],
             [0.6, '#fee08b'],
             [0.7, '#fdae61'],
             [0.8, '#f46d43'],
             [0.9, '#d73027'],
             [1.0, '#a50026']]

    #color_values = mapper.project(np.array(shape_df), projection=[var_to_index['COLOR']])
    color_values = np.array(shape_df['COLOR'])
    color_function_name = ['State Color']
    my_colorscale = pl_brewer
    kmgraph,  mapper_summary, colorf_distribution = get_mapper_graph(scomplex, 
                                                                     color_values=color_values,  
                                                                     color_function_name=color_function_name, 
                                                                     colorscale=my_colorscale)

    bgcolor = 'rgba(10,10,10, 0.9)'
    # y_gridcolor = 'rgb(150,150,150)'# on a black background the gridlines are set on  grey

    plotly_graph_data = plotly_graph(kmgraph, graph_layout='fr', colorscale=my_colorscale, 
                                     factor_size=2.5, edge_linewidth=0.5)
    plot_title = "States"
    # plot_title = 'Pool 13, Summer 1993-1999; Epsilon ' + str(DBSCAN_EPSILON) + ', MIN_SAMPLES ' + str(DBSCAN_MIN_SAMPLES) 
    layout = plot_layout(title=plot_title,  
                         width=620, height=570,
                         annotation_text=get_kmgraph_meta(mapper_summary),  
                         bgcolor=bgcolor)

    # FigureWidget is responsible for event listeners

    fw_graph = go.FigureWidget(data=plotly_graph_data, layout=layout)
    fw_hist = node_hist_fig(colorf_distribution, bgcolor=bgcolor)
    fw_summary = summary_fig(mapper_summary, height=300)

    dashboard = hovering_widgets(kmgraph, 
                                 fw_graph, 
                                 bgcolor=bgcolor, 
                                 member_textbox_width=600)

    # DESIRED FILE PATH, CHANGE TO FIT YOUR LOCAL MACHINE
    directory_path = "Users\\killiad\\Documents\\Senior\\REU\\Mapper\\PCA_1_NO_COND\\"
    
    #Update the fw_graph colorbar, setting its title:
    fw_graph.data[1].marker.colorbar.title = 'dist to<br>x-min'
    html_output_path = directory_path + "States_" + 'PCA_1' + 'all_var_' + '.html'
    html_output_path = html_output_path.replace(" ", "-")
    html_output_path = html_output_path.replace(":", "_")
    html_output_path = "C:\\" + html_output_path
    mapper.visualize(scomplex, color_values=color_values, color_function_name=color_function_name, path_html=html_output_path,
                    lens = summary_variable, lens_names = projected_vars)

In [11]:
mapper(scomplex, shape_df)